In [1]:
import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn import functional as F
from sample_data.Model_8 import Net
from sample_data.data import get_dataloaders
from torchsummary import summary
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

# For reproducibility
torch.manual_seed(1)

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

if cuda:
    torch.cuda.manual_seed(1)

# dataloader arguments
dataloader_args = dict(shuffle=True, batch_size=32, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64, num_workers=0, pin_memory=False)



# Get train dataloader and train data
data_dir = './data'
train_loader, test_loader, exp_data_train, exp_test,test_loader = get_dataloaders(data_dir, dataloader_args['batch_size'], dataloader_args['num_workers'], dataloader_args['pin_memory'])


# Test dataloader (fix the error related to the test dataset)
#test_transforms = transforms.Compose([transforms.ToTensor(),])
#exp_test = datasets.MNIST(data_dir, train=False, download=True, transform=test_transforms)
#test_loader = torch.utils.data.DataLoader(exp_test, **dataloader_args)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

model = Net().to(device)
summary(model, input_size=(1, 28, 28))

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#scheduler = StepLR(optimizer, step_size=6, gamma=0.1)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

EPOCHS = 16
train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        y_pred = model(data)
        loss = F.nll_loss(y_pred, target)
        train_losses.append(loss)

        loss.backward()
        optimizer.step()

        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc=f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100 * correct / processed:0.2f}')
        train_acc.append(100 * correct / processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.2f}%)\n')

    test_acc.append(100. * correct / len(test_loader.dataset))


for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

# Print train and test accuracy
print("\nTrain Accuracy:", train_acc[-1])
print("Test Accuracy:", test_acc[-1])


CUDA Available? True
cuda


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
       BatchNorm2d-2            [-1, 8, 26, 26]              16
         LeakyReLU-3            [-1, 8, 26, 26]               0
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
       BatchNorm2d-6           [-1, 16, 24, 24]              32
         LeakyReLU-7           [-1, 16, 24, 24]               0
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9            [-1, 8, 24, 24]             128
        MaxPool2d-10            [-1, 8, 12, 12]               0
           Conv2d-11           [-1, 12, 10, 10]             864
      BatchNorm2d-12           [-1, 12, 10, 10]              24
        LeakyReLU-13           [-1, 12, 10, 10]               0
          Dropout-14           [-1, 12,

Loss=0.03617025539278984 Batch_id=1874 Accuracy=93.24: 100%|██████████| 1875/1875 [00:25<00:00, 73.75it/s]



Test set: Average loss: 0.0902, Accuracy: 9732/10000 (97.32%)

EPOCH: 1


...


Test set: Average loss: 0.0206, Accuracy: 9937/10000 (99.37%)

EPOCH: 14


Loss=0.01901928149163723 Batch_id=1874 Accuracy=99.41: 100%|██████████| 1875/1875 [00:22<00:00, 83.20it/s]



Test set: Average loss: 0.0201, Accuracy: 9936/10000 (99.36%)

EPOCH: 15


Loss=0.02643531560897827 Batch_id=1874 Accuracy=99.41: 100%|██████████| 1875/1875 [00:22<00:00, 83.56it/s]



Test set: Average loss: 0.0218, Accuracy: 9932/10000 (99.32%)


Train Accuracy: 99.40666666666667
Test Accuracy: 99.32
